In [4]:
import os
import json
from neo4j import GraphDatabase

# Label mapping for node types
label_map = {
    "character": "Human",
    "ai": "AI",
    "organization": "Organization",
    "location": "Location",
    "weapon": "Weapon",
    "vehicle": "Vehicle",
    "timelines": "Timeline"
}

# Connect to Neo4j
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))

# Create entity node
def create_entity(tx, name, label):
    cypher_label = label_map.get(label, "Entity")
    query = f"""
    MERGE (n:{cypher_label} {{name: $name}})
    SET n.type = $label_type, n.confidence = $confidence
    """
    tx.run(query, name=name, label_type=label.lower(), confidence=0.9)


# Create relationship (assumes subject & object nodes exist or are already merged)
def create_relation(tx, subject, predicate, obj, release_date, movie_title):
    # Safely handle predicates with special characters by wrapping them in backticks
    predicate = f"`{predicate.upper().replace(' ', '_')}`"
    
    # The query now includes both release_date and movie_title as properties on the relationship
    query = f"""
    MERGE (a {{name: $subject}})
    MERGE (b {{name: $object}})
    MERGE (a)-[r:{predicate}]->(b)
    SET r.release_date = $release_date,
        r.movie_title = $movie_title
    """

    tx.run(query, subject=subject, object=obj, release_date=release_date, movie_title=movie_title)


# Iterate over the JSON files in the 'final' folder
final_folder = "final"
for filename in os.listdir(final_folder):
    if filename.endswith(".json"):
        file_path = os.path.join(final_folder, filename)
        
        # Load data from each JSON file
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        entities = data["entities"]
        relations = data["relations"]

        # Extract the movie title for logging
        movie_title = data.get("title", "Unknown")

        print(f"Processing movie: {movie_title}...")

        # Load Entities and Relations into Neo4j
        with driver.session(database="final") as session:
            # Load Entities
            for entity in entities:
                session.write_transaction(create_entity, entity["text"], entity["label"])

            # Load Relations (no validation)
            for rel in relations:
                session.write_transaction(create_relation, rel["subject"], rel["predicate"], rel["object"],data['release_date'],data['title'])

        print(f"✅ Processed and saved entities and relations for {movie_title}")

# Close Neo4j connection
driver.close()

print("All movies processed and saved to Neo4j successfully!")


Processing movie: Agent Carter...


C:\Users\Thakk\AppData\Local\Temp\ipykernel_18000\4024723300.py:68: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_entity, entity["text"], entity["label"])
C:\Users\Thakk\AppData\Local\Temp\ipykernel_18000\4024723300.py:72: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relation, rel["subject"], rel["predicate"], rel["object"],data['release_date'],data['title'])


✅ Processed and saved entities and relations for Agent Carter
Processing movie: All Hail the King...
✅ Processed and saved entities and relations for All Hail the King
Processing movie: Ant-Man...
✅ Processed and saved entities and relations for Ant-Man
Processing movie: Ant-Man & The Wasp...
✅ Processed and saved entities and relations for Ant-Man & The Wasp
Processing movie: Avengers: Age of Ultron...
✅ Processed and saved entities and relations for Avengers: Age of Ultron
Processing movie: Avengers: Assemble...
✅ Processed and saved entities and relations for Avengers: Assemble
Processing movie: Avengers: Endgame...
✅ Processed and saved entities and relations for Avengers: Endgame
Processing movie: Avengers: Infinity War...
✅ Processed and saved entities and relations for Avengers: Infinity War
Processing movie: A Funny Thing Happened on the Way to Thor's Hammer...
✅ Processed and saved entities and relations for A Funny Thing Happened on the Way to Thor's Hammer
Processing movie: 